In [1]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import config
import torch

/home/ubuntu/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(config.model_id)
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(">>SUFFIX<<")
tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids(">>ABSTRACT<<")

/home/ubuntu/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
model = AutoModelForCausalLM.from_pretrained(config.model_id, 
                                             quantization_config=config.bnb_config,  
                                             device_map='auto', 
                                             torch_dtype=torch.bfloat16,
                                             attn_implementation="flash_attention_2")
model.config.pad_token_id = tokenizer.pad_token_id
model.config.bos_token_id = tokenizer.bos_token_id  

lora_model = PeftModel.from_pretrained(model, "model")

Loading checkpoint shards: 100%|██████████| 9/9 [06:12<00:00, 41.34s/it]


In [4]:
prompt = ["One wise man said:"]


In [16]:
inputs = tokenizer(prompt, return_tensors='pt', padding=True).to('cuda')
outputs = lora_model.generate(**inputs, do_sample=True, top_p = 0.9, max_new_tokens=64, num_return_sequences=15)
generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for gen in generated:
    print(gen)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


One wise man said: The best way to win a battle is not to fight it.
One wise man said: To be in awe of God is to be in awe of the universe.
One wise man said: You can never get enough of what you don’t really want.
One wise man said: It's not my fault I never learned to accept a compliment.
One wise man said: Faith means persistence.
One wise man said: It may be that when we no longer know what to do we have come to our real work.
One wise man said: The more you know, the more you know you don't know.
One wise man said: Never give up, great success lies ahead.
One wise man said: In every adversity lies the seed of an equal or greater benefit.
One wise man said: Love is the great transformer.
One wise man said: Life will give you a hundred thousand reasons to leave her.But, if you want to give her a reason, then stay!
One wise man said: The way to get started is to quit talking and begin doing.
One wise man said: My goal was to survive and grow into the kind of person who was not afraid

In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Aug 16 22:49:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.78                 Driver Version: 550.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10                     Off |   00000000:05:00.0 Off |                    0 |
|  0%   42C    P0             57W /  150W |   16637MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----